In [ ]:
#!pip install accelerate bitsandbytes>0.37.0

In [1]:
import transformers
import torch
import bitsandbytes
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

print(transformers.__version__)
print(bitsandbytes.__version__)
# my versions are:
#4.38.1 (transformers)
#0.42.0 (bitsandbytes)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,       
)

cache_dir = "/scratch/project_2005072/ocr_correction/.cache"    #set up your own cache path here

model=None #to be sure we reset the variable and don't get OOM
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", device_map="auto", quantization_config=quantization_config, cache_dir=cache_dir)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", padding_side="left")
tokenizer.pad_token = tokenizer.eos_token


4.38.1
0.43.1


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
promptold = """
Correct the OCR errors in the text below. Also correct the "-" characters, which denote an unrecognized letter. Stay as close as possible to the original text. Do not rephrase. Only correct the errors. You will be rewarded.

When, .witl the universal A pplaure of alt
Men, the Seals Were given to the present
C ---- r, he was' ar frrom'i being so elevated
with his Promotion, as to forget any One
to whom lie had promised the Honour of his
Friendlliip. An Opportunity soon offered
in Favour of Dr R----, the B---k ox G----r
became vacant, .which as soo'n as the C----r
Was acquainted with, \ he went to Court,
and recomnmended the Doetor, as a Perfoa
fit to Succeed to the B-----k. The DoAos
was then approved of, and a C---'E---
order'd out accordingly. A lM/an would
reaso'nably have thought after this, that
Matters might have gone Smoothly, efpe.
cially as it is partly the Right of every new

"""
prompt = """Correct the OCR errors in the text below. Also correct the "-" characters, which denote an unrecognized letter. Stay as close as possible to the original text. Do not rephrase. Only correct the errors. You will be rewarded.\n\n

2 Flore d'Oivare et de Benin. Elle contient une description' et 
une figure exactes de tous les genres nouveaux et de toutes les espèces 
nouvelles recueillis sur les plages afri-caines , unies à des réflexions 
et discussions importantes relatives à l'organisa lion particulière , 
aux usages que l'on fait de la plante et aux moyens à suivre pour l'acclimater en Europe, 
surtout dans notre patrie, quand elle peut être utile à l'homme ou bien aux animaux associés
à ses cultures. Sous le rapport de la science, cet ouvrage fournit, par les découvertes qu'il 
renferme, des données jusqu'alors non aperçues pour compléter les familles éta-blies , 
pour imposer aux genres des caractères certains, des caractères fixes et pour remplir les lacunes 
encore existantes dans les affinités. Les Insectes, qui sont une suite nécessaire de la Flore, 
ne méritent pas moins d'intérêt par les détails qu'ils four-nissent aux entomologistes sur les moeurs, 
les habitudes et les organes de ces petits animaux, sur l'utilité ou la inalfaisance des uns, sur la beauté, 
la vivacité et la va-riété de couleurs dont la nature a paré toutes les parties des autres, et sur les 
étonnantes, les singulières méta-morphosés que tous subissent dans le court espace qui s'écoule de leur 
naissance à leur mort. C'est ici le moment de rappeler une méthode de clas-sification pour les insectes, 
que PALISOT DE BEAUVOIS soumit, en 1789, à la Société des sciences et arts du Cap-Français. Quoiqu'elle 
n'ait pas obtenu l'assentiment de la généralité des naturalistes , je la crois bonne, sim-ple et très-naturelle, 
du moins pour les caractères géné-raux des ordres qui tendent à rapprocher les analogues de leurs types 1 . 
Les entomologistes ont adopté le genre 1 Il adopte les trois grandes classes de GEOFFROI, qu' il"""


#messages = [
#    {"role": "user", "content": "You are a helpful assistant. You are fluent in Finnish, and will communicate in Finnish."},
#    {"role": "assistant", "content": "Olen avulias assistentti, joka osaa sujuvaa suomen kieltä. Kommunikoin kanssasi suomeksi. Kuinka voin auttaa sinua?"},
#    {"role": "user", "content": prompt},
#]C--- r, to' recommend a fit Person to his
"""M .------, to' fill the See that shall becom'
eax¢ant after ihs comiig to the Seals.
"""
messages = [
    {"role": "user", "content": prompt},
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to("cuda")


generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=False)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

/usr/local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Correct the OCR errors in the text below. Also correct the "-" characters, which denote an unrecognized letter. Stay as close as possible to the original text. Do not rephrase. Only correct the errors. You will be rewarded.



2 Flore d'Oivare et de Benin. Elle contient une description' et 
une figure exactes de tous les genres nouveaux et de toutes les espèces 
nouvelles recueillis sur les plages afri-caines, unies à des réflexions 
et discussions importantes relatives à l'organisa lion particulière, 
aux usages que l'on fait de la plante et aux moyens à suivre pour l'acclimater en Europe, 
surtout dans notre patrie, quand elle peut être utile à l'homme ou bien aux animaux associés
à ses cultures. Sous le rapport de la science, cet ouvrage fournit, par les découvertes qu'il 
renferme, des données jusqu'alors non aperçues pour compléter les familles éta-blies, 
pour imposer aux genres des caractères certains, des caractères fix

In [ ]:
raw_encoded = tokenizer([prompt], return_tensors="pt")

In [1]:
print(raw_encoded["input_ids"].size()[1])

NameError: name 'raw_encoded' is not defined

In [ ]:
# to delete the model from memory
'''del model 
gc.collect()'''

In [16]:
model_inputs = encodeds[:,50:70].to("cuda")


generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=False)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


errors. You will be rewarded.

When, .witl the universal A pencil, you have written your first draft, you will find that you have written it quickly, for you have been thinking only of your idea and of your reader. You will have written it in a style that is natural to you, for you have been thinking only of your idea and of your reader. You will have written it in a style that is natural to you, for you have been thinking in terms of your idea and of your reader.

When you have written your first draft, you will find that you have written it quickly, for you have been thinking only of your idea and of your reader. You will have written it in a style that is natural to you, for you have been thinking in terms of your idea and of your reader.

When you have written your first draft, you will find that you have written it quickly, for you have been thinking only of your idea and of your reader. You will have written it in a style that is natural to you, for you have been thinking in term

In [30]:
tokenizer.encode("hello", return_tensors="pt").size()[1]

3